In [2]:
from pathlib import Path
import pandas as pd


In [3]:
url_df = 'https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures'

df = pd.read_html(url_df)

HTTPError: HTTP Error 429: Too Many Requests

In [3]:
df = pd.read_html(url_df)[0]
df.head()
df.columns = [' '.join(col).strip() for col in df.columns]

df = df.reset_index(drop=True)
df.head()

,W k,D a y,D a t e,T i m e,H o m e,x G,S c o r e,x G . 1,A w a y,A t t e n d a n c e,V e n u e,R e f e r e e,M a t c h R e p o r t,N o t e s
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN


In [4]:
df.columns = [col.replace(" ", "") for col in df.columns]
df = df.dropna(how='all')
df = df.drop('MatchReport', axis=1)
df = df.drop('Notes', axis=1)

In [5]:
df['Split Score'] = df['Score'].str.split('–')
df['Home Score'] = df['Split Score'].str[0]
df['Away Score'] = df['Split Score'].str[1]
df = df.drop('Split Score', axis=1)

In [6]:
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Home Score,Away Score
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,0,3
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,2,1
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,0,1
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,0,1
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,38.0,Sun,2024-05-19,16:00,Brentford,1.1,2–4,3.4,Newcastle Utd,17124.0,Gtech Community Stadium,Simon Hooper,2,4
419,38.0,Sun,2024-05-19,16:00,Chelsea,1.1,2–1,2.2,Bournemouth,39724.0,Stamford Bridge,Anthony Taylor,2,1
420,38.0,Sun,2024-05-19,16:00,Crystal Palace,2.5,5–0,0.9,Aston Villa,25191.0,Selhurst Park,Darren Bond,5,0
421,38.0,Sun,2024-05-19,16:00,Liverpool,4.5,2–0,0.5,Wolves,60059.0,Anfield,Chris Kavanagh,2,0


In [7]:
def get_winner(row):
    # Replace any non-standard dash with a standard hyphen and split scores
    home_score = row['Home Score']
    away_score = row['Away Score']    
    
    # Determine the winner
    if home_score > away_score:
        return row["Home"]
    elif home_score < away_score:
        return row["Away"]
    else:
        return "Draw"
    
# Apply the function to each row and create a new column 'Winner'
df["Winner"] = df.apply(get_winner, axis=1)

In [8]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Home Score,Away Score,Winner
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,0,3,Manchester City
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,2,1,Arsenal
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,0,1,Fulham
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,0,1,Crystal Palace
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,4,1,Brighton


In [77]:
df.iloc[10]

Wk                         2.0
Day                        Fri
Date                2023-08-18
Time                     19:45
Home           Nott'ham Forest
xG                         1.4
Score                      2–1
xG.1                       0.5
Away             Sheffield Utd
Attendance             29432.0
Venue          The City Ground
Referee           Peter Bankes
Split Score             [2, 1]
Home Score                   2
Away Score                   1
Winner         Nott'ham Forest
Name: 11, dtype: object

In [78]:
filepath = Path('2024.csv')  
df.to_csv(filepath, index=False)